## Contributions  
A.Carolina Gonzalez: **CG** Federico Zampa: **FZ** Nina Shi: **NS** <br>


Sort dataframe: CG,NS,FZ<br>
Calculate mean value of replicates: NS<br>
Calculate sd and z-score values: CG<br>
Shape dataframe for heatmap: CG,NS<br>
Setup heatmap: FZ

In [2]:
#Import Library 
library(tidyverse)
library(reshape2)
library(gplots)

In [3]:
#import data file
countData <- read.csv(file="PMID-26950239-8Lib-log2RPKM.csv", header = TRUE)

#Sort genes of interest 
genes <- c("Gzma","Cx3cr1","Sema4a","Ccr5","Prf1","Aqp9","Cercam","Zeb2","Dok2","Klrg1",
            "S1pr5","Il18rap","Gzmk","Serpinb9","Klrk1","Tbx21","Zfp683","Ctla2a","Gzmm","Il12rb2",
            "Gzmb","Havcr2","Fasl","Id2","Ccr2","Ifng","Cxcr6","Cish","Il7r","Tnfrsf9",
            "Ccr6","Egr1","Egr2","Lag3","Itgae","Icos","Hif1a","Tspan32","Tnfrsf4","Batf",
            "Bcl6","Btla","Pou2af1","Irf4","Il1r2","Ltb","Tnf","Il23r","Ccr7","Cxcr5",
            "Id3","Lta","Tox","Tcf7","Batf3","Maf","Ccr9","Il17a","Rorc") 

#Return matches of the "genes" vector in the "Symbol" column of countData 
countData <- countData[match(genes, countData$Symbol),] 

head(countData)


,Symbol,S1.WT.1,S5.WT.2,S2.BlimpKO.1,S6.BlimpKO.2,S3.TbetKO.1,S7.TbetKO.2,S4.DKO.1,S8.DKO.2
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3624,Gzma,9.600537,9.243791,7.590605,6.732208,5.3525210,7.819147,3.809018,3.1152665
13490,Cx3cr1,7.645995,7.624803,6.128040,5.536862,4.5084692,6.278160,3.432644,2.4403507
8127,Sema4a,7.269906,7.219532,6.369266,6.328405,6.1241316,6.443113,4.953529,4.9280705
13538,Ccr5,6.844842,6.756816,5.644583,5.326466,5.0132919,5.744329,3.479213,3.1641177
997,Prf1,8.449985,8.245118,7.087723,6.888363,6.9727462,7.789022,5.411753,5.3942794
13190,Aqp9,3.038762,2.955309,1.019415,2.100406,0.9734355,1.872903,-0.506833,0.6648396


In [4]:
#Calculate mean of samples replicates 

#countData_mean have 4 columns and same number of rows as countData
#[x,y] represent the location in mean dataframe, x indicates the row, y indicates the column
#for each gene located at xth row in countData_mean,
#the corresponding duplicates is located at [x,2*y] and [x,2*y+1] in the countData
#y=1 calculate the mean of WT, y=2 calculate the mean of BlimpKO
#y=3 calculate the mean of TbetKO, y=4 calculate the mean of DKO
countData_mean <- data.frame()
for (y in 1:4){
    for (x in 1:nrow(countData)){
        countData_mean[x,y] <- mean(c(countData[x,2*y],countData[x,2*y+1]))
    }
}




#rename column as sample name
countData_mean <- rename(countData_mean,
    "WT" = V1,
    "Blimp1fl/flLck-Cre" = V2, #BlimpKO
    "Tbx21−/−" = V3, #TbetKO
    "Tbx21−/−Blimp1fl/flLck-Cre" = V4) #DKO

head(countData_mean)
                   

,WT,Blimp1fl/flLck-Cre,Tbx21−/−,Tbx21−/−Blimp1fl/flLck-Cre
,<dbl>,<dbl>,<dbl>,<dbl>
1,9.422164,7.161406,6.585834,3.46214223
2,7.635399,5.832451,5.393315,2.93649757
3,7.244719,6.348835,6.283622,4.94079990
4,6.800829,5.485525,5.378810,3.32166556
5,8.347552,6.988043,7.380884,5.40301631
6,2.997036,1.559910,1.423169,0.07900328


In [5]:
#Calculate standard deviation using "sd" function. Create a new column with the sd value

countData_mean$sd_data <- apply(countData_mean,1,sd) 

head(countData_mean)


,WT,Blimp1fl/flLck-Cre,Tbx21−/−,Tbx21−/−Blimp1fl/flLck-Cre,sd_data
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,9.422164,7.161406,6.585834,3.46214223,2.4571485
2,7.635399,5.832451,5.393315,2.93649757,1.9359014
3,7.244719,6.348835,6.283622,4.94079990,0.9497519
4,6.800829,5.485525,5.378810,3.32166556,1.4370763
5,8.347552,6.988043,7.380884,5.40301631,1.2258192
6,2.997036,1.559910,1.423169,0.07900328,1.1928909


In [6]:
#Calculate the ZSCORE 

#Formula: x-mean/sd
#x: each gene located in countData_mean 
#Mean:Mean of each row. Calulate the value using rowMeans(). 
#sd: standard deviation calculated


zscore_values <- (countData_mean[,1:4]- rowMeans(countData_mean[,1:4]))/ countData_mean$sd_data  #1:4 indicates the sample columns


head (zscore_values)


,WT,Blimp1fl/flLck-Cre,Tbx21−/−,Tbx21−/−Blimp1fl/flLck-Cre
,<dbl>,<dbl>,<dbl>,<dbl>
1,1.124994,0.20492035,-0.02932369,-1.300591
2,1.129181,0.19785900,-0.02897913,-1.298061
3,1.095259,0.15197778,0.08331467,-1.330552
4,1.081447,0.16618270,0.09192479,-1.339554
5,1.074936,-0.03412472,0.28634771,-1.327159
6,1.242575,0.03783308,-0.07679705,-1.203611


In [7]:
#Assign gene names to 1st column
Gene = c("Gzma","Cx3cr1","Sema4a","Ccr5","Prf1","Aqp9","Cercam","Zeb2","Dok2","Klrg1",
            "S1pr5","Il18rap","Gzmk","Serpinb9","Klrk1","Tbx21","Zfp683","Ctla2a","Gzmm","Il12rb2",
            "Gzmb","Havcr2","Fasl","Id2","Ccr2","Ifng","Cxcr6","Cish","Il7r","Tnfrsf9",
            "Ccr6","Egr1","Egr2","Lag3","Itgae","Icos","Hif1a","Tspan32","Tnfrsf4","Batf",
            "Bcl6","Btla","Pou2af1","Irf4","Il1r2","Ltb","Tnf","Il23r","Ccr7","Cxcr5",
            "Id3","Lta","Tox","Tcf7","Batf3","Maf","Ccr9","Il17a","Rorc") #List of genes 
 
zscore_values <- cbind(Gene,zscore_values) #Bind Gene list to zscore_values data frame


head(zscore_values)


,Gene,WT,Blimp1fl/flLck-Cre,Tbx21−/−,Tbx21−/−Blimp1fl/flLck-Cre
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
1,Gzma,1.124994,0.20492035,-0.02932369,-1.300591
2,Cx3cr1,1.129181,0.19785900,-0.02897913,-1.298061
3,Sema4a,1.095259,0.15197778,0.08331467,-1.330552
4,Ccr5,1.081447,0.16618270,0.09192479,-1.339554
5,Prf1,1.074936,-0.03412472,0.28634771,-1.327159
6,Aqp9,1.242575,0.03783308,-0.07679705,-1.203611


In [9]:
#Reshape data using melt function
df_heatmap <- melt(zscore_values)

#Rename columns of new melt df_heatmap 
df_heatmap <-  rename(df_heatmap,
    Sample = variable, 
  z_score = value)

head(df_heatmap)


Using Gene as id variables



,Gene,Sample,z_score
,<fct>,<fct>,<dbl>
1,Gzma,WT,1.124994
2,Cx3cr1,WT,1.129181
3,Sema4a,WT,1.095259
4,Ccr5,WT,1.081447
5,Prf1,WT,1.074936
6,Aqp9,WT,1.242575


In [10]:
#reshape data to matrix for heatmap.2
hm_matrix <- data.matrix(zscore_values)
row.names(hm_matrix) <- zscore_values$Gene #set the gene name as row name
hm_matrix <- hm_matrix[,-1] # remove gene column

head(hm_matrix)


,WT,Blimp1fl/flLck-Cre,Tbx21−/−,Tbx21−/−Blimp1fl/flLck-Cre
Gzma,1.124994,0.20492035,-0.02932369,-1.300591
Cx3cr1,1.129181,0.19785900,-0.02897913,-1.298061
Sema4a,1.095259,0.15197778,0.08331467,-1.330552
Ccr5,1.081447,0.16618270,0.09192479,-1.339554
Prf1,1.074936,-0.03412472,0.28634771,-1.327159
Aqp9,1.242575,0.03783308,-0.07679705,-1.203611


In [11]:
#Heatmap

#Costumize rows
redcols <- as.list(c("Ccr6", "Il23r","Il17a","Rorc"))   #define list with gene names to highlight in red
cols <- rep('black', nrow(hm_matrix))   #vector where all rownames are colored in black
cols[row.names(hm_matrix) %in% redcols] <- 'red'   # color in red the specified rownames from redcols

#  define color breaks and transitions
my_palette <- colorRampPalette(c("black", "blue", "yellow"))(n=299)  

# create PNG filename for the heat map
png("heatmap_Capstone_Team1.png",            
  width = 5*300,        # 5 x 300 pixels
  height = 5*300,
  res = 300,            # 300 pixels per inch
  pointsize = 7)        # font size

#Set up heatmap
heatmap.2(hm_matrix,
          density.info="none",  # turns off density plot inside color legend
          trace="none",         # turns off trace lines inside the heat map
          dendrogram='none',     # turns off dendrogram
          col=my_palette,  # use previously defined color palette
          colRow = cols, # assign red text to previously selected genes
          lmat=rbind( c(3, 4), c(2,1) ), # create matrix to put the key on top of heatmap, number meaning: 1=Heatmap,2=Row dendrogram,3=Column dendrogram,4=Key
          lhei=c(1.2, 4), # height
          lwid=c(1.5, 1), # width 
          key.xlab="Row z-score",  # key title 
          key.title=NA,    # remove original key title
          key.par=list(mgp=c(-3, 0.5, 0), mar=c(2,0,10,10)),   # dimensions/positioning of key parts
          margins=c(7, 9),  # respectively botton and left margins
          Colv=FALSE, # use our defined column ordering
          Rowv=FALSE, # use our defined row ordering
          srtCol=45, # angle column label text
          cexCol=0.6, # size column label text
          cexRow=0.5,  # size row label text
          offsetRow=-0.6, # position row text closer to heatmap
          offsetCol=-0.25)  # position column text closer to heatmap
dev.off() #close the figure file


Warning message in (function (..., no.readonly = FALSE) :
“`mgp[1:3]' are of differing sign”


png 
  2